In [1]:
import pandas as pd
import requests
import numpy as np
import re
import os
import json
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from geopy import distance
import openrouteservice as ors
import time

SA2

In [2]:
SA2_shape = gpd.read_file("../data/landing/SA2-shapefile/")
# SA2_shape

SA2_inf = ['SA2_CODE21', 'SA2_NAME21', 'geometry']
SA2_shape = SA2_shape[SA2_inf]
SA2_shape

,SA2_CODE21,SA2_NAME21,geometry
0,101021007,Braidwood,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,"POLYGON ((149.21899 -35.36738, 149.218 -35.366..."
2,101021009,Queanbeyan,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,"POLYGON ((149.19572 -35.36126, 149.1997 -35.35..."
...,...,...,...
2468,901031003,Jervis Bay,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),None
2471,999999499,No usual address (OT),None


In [3]:
SA2_shape.crs = 'EPSG: 4326'
SA2_shape

,SA2_CODE21,SA2_NAME21,geometry
0,101021007,Braidwood,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,"POLYGON ((149.21899 -35.36738, 149.218 -35.366..."
2,101021009,Queanbeyan,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,"POLYGON ((149.19572 -35.36126, 149.1997 -35.35..."
...,...,...,...
2468,901031003,Jervis Bay,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),None
2471,999999499,No usual address (OT),None


Income

Population

School Zone

PTV

In [13]:
shape_1 = pd.read_csv("../data/landing/ptv/1/1/shapes.txt")
shape_1


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,1-ABY-mjp-10.1.H,-36.084262,146.924527,1,0.00
1,1-ABY-mjp-10.1.H,-36.085028,146.924381,2,86.20
2,1-ABY-mjp-10.1.H,-36.088625,146.923734,3,490.37
3,1-ABY-mjp-10.1.H,-36.090905,146.923265,4,747.44
4,1-ABY-mjp-10.1.H,-36.091681,146.923064,5,835.51
...,...,...,...,...,...
869753,1-WBL-mjp-9.5.R,-37.811439,144.945707,1352,274128.87
869754,1-WBL-mjp-9.5.R,-37.813874,144.948934,1353,274520.89
869755,1-WBL-mjp-9.5.R,-37.814151,144.949289,1354,274564.74
869756,1-WBL-mjp-9.5.R,-37.816379,144.950966,1355,274852.95


In [15]:
stops_1 = pd.read_csv("../data/landing/ptv/1/1/stops.txt")
stops_1 = stops_1.loc[:,["stop_name","stop_lat","stop_lon"]]

In [16]:
stops_1

,stop_name,stop_lat,stop_lon
0,Wallan Railway Station (Wallan),-37.416861,145.005372
1,Melton Railway Station (Melton South),-37.703359,144.572216
2,Rockbank Railway Station (Rockbank),-37.729261,144.650631
3,Deer Park Railway Station (Deer Park),-37.777764,144.772304
4,Sunbury Railway Station (Sunbury),-37.579206,144.728165
...,...,...,...
104,Cobblebank Railway Station (Cobblebank),-37.712546,144.604108
105,Raywood Railway Station (Raywood),-36.531959,144.201161
106,Huntly Railway Station (Huntly),-36.665848,144.369820
107,Goornong Railway Station (Goornong),-36.615183,144.503474


In [17]:
gdf_stops_1 = gpd.GeoDataFrame(
    stops_1, geometry=gpd.points_from_xy(stops_1['stop_lon'], stops_1['stop_lat']))

In [18]:
gdf_stops_1

,stop_name,stop_lat,stop_lon,geometry
0,Wallan Railway Station (Wallan),-37.416861,145.005372,POINT (145.00537 -37.41686)
1,Melton Railway Station (Melton South),-37.703359,144.572216,POINT (144.57222 -37.70336)
2,Rockbank Railway Station (Rockbank),-37.729261,144.650631,POINT (144.65063 -37.72926)
3,Deer Park Railway Station (Deer Park),-37.777764,144.772304,POINT (144.7723 -37.77776)
4,Sunbury Railway Station (Sunbury),-37.579206,144.728165,POINT (144.72816 -37.57921)
...,...,...,...,...
104,Cobblebank Railway Station (Cobblebank),-37.712546,144.604108,POINT (144.60411 -37.71255)
105,Raywood Railway Station (Raywood),-36.531959,144.201161,POINT (144.20116 -36.53196)
106,Huntly Railway Station (Huntly),-36.665848,144.369820,POINT (144.36982 -36.66585)
107,Goornong Railway Station (Goornong),-36.615183,144.503474,POINT (144.50347 -36.61518)


In [19]:
gdf_stops_1.crs = 'EPSG: 4326'